![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/tutorials/quickstart-ci/AzureMLin10mins.png)

# Quickstart: Train and deploy a model in Azure Machine Learning in 10 minutes

In this quickstart, learn how to get started with Azure Machine Learning. You'll train an image classification model using the [MNIST](https://azure.microsoft.com/services/open-datasets/catalog/mnist/) dataset.

You'll learn how to:

> * Download a dataset and look at the data
> * Train an image classification model and log metrics
> * Deploy the model

## Connect to your workspace and create an experiment

Import some libraries and create an experiment to track the runs in your workspace. A workspace can have multiple experiments, and all  users that have access to the workspace can collaborate on them.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import azureml.core
from azureml.core import Workspace
from azureml.core import Experiment

# connect to your workspace
ws = Workspace.from_config()

# create experiment and start logging to a new run in the experiment
experiment_name = "azure-ml-in10-mins-tutorial"
exp = Experiment(workspace=ws, name=experiment_name)
run = exp.start_logging(snapshot_directory=None)

## Import Data

Before you train a model, you need to understand the data you're using to train it. In this section, learn how to:

* Download the MNIST dataset
* Display some sample images

### Download the MNIST dataset

You'll use Azure Open Datasets to get the raw MNIST data files. [Azure Open Datasets](https://docs.microsoft.com/azure/open-datasets/overview-what-are-open-datasets) are curated public datasets that you can use to add scenario-specific features to machine learning solutions for better models. Each dataset has a corresponding class, `MNIST` in this case, to retrieve the data in different ways.

In [ ]:
import os
from azureml.core import Dataset
from azureml.opendatasets import MNIST

data_folder = os.path.join(os.getcwd(), "data")
os.makedirs(data_folder, exist_ok=True)

mnist_file_dataset = MNIST.get_file_dataset()
mnist_file_dataset.download(data_folder, overwrite=True)

mnist_file_dataset = mnist_file_dataset.register(
    workspace=ws,
    name="mnist_opendataset",
    description="training and test dataset",
    create_new_version=True,
)

### Take a look at the data

Load the compressed files into `numpy` arrays. Then use `matplotlib` to plot 30 random images from the dataset with their labels above them. 

Note this step requires a `load_data` function that's included in an `utils.py` file. This file is placed in the same folder as this notebook. The `load_data` function simply parses the compressed files into numpy arrays.

In [ ]:
from utils import load_data
import matplotlib.pyplot as plt
import numpy as np
import glob


# note we also shrink the intensity values (X) from 0-255 to 0-1. This helps the model converge faster.
X_train = (
    load_data(
        glob.glob(
            os.path.join(data_folder, "**/train-images-idx3-ubyte.gz"), recursive=True
        )[0],
        False,
    )
    / 255.0
)
X_test = (
    load_data(
        glob.glob(
            os.path.join(data_folder, "**/t10k-images-idx3-ubyte.gz"), recursive=True
        )[0],
        False,
    )
    / 255.0
)
y_train = load_data(
    glob.glob(
        os.path.join(data_folder, "**/train-labels-idx1-ubyte.gz"), recursive=True
    )[0],
    True,
).reshape(-1)
y_test = load_data(
    glob.glob(
        os.path.join(data_folder, "**/t10k-labels-idx1-ubyte.gz"), recursive=True
    )[0],
    True,
).reshape(-1)


# now let's show some randomly chosen images from the traininng set.
count = 0
sample_size = 30
plt.figure(figsize=(16, 6))
for i in np.random.permutation(X_train.shape[0])[:sample_size]:
    count = count + 1
    plt.subplot(1, sample_size, count)
    plt.axhline("")
    plt.axvline("")
    plt.text(x=10, y=-10, s=y_train[i], fontsize=18)
    plt.imshow(X_train[i].reshape(28, 28), cmap=plt.cm.Greys)
plt.show()

## Train model and log metrics

You'll train the model using the code below. Your training runs and metrics will be registered in the experiment you created, so that this information is available after you've finished.

You'll be using the [LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) classifier from the [SciKit Learn framework](https://scikit-learn.org/) to classify the data.

> **Note: The model training takes around 1 minute to complete.**

In [ ]:
# create the model
import numpy as np
from sklearn.linear_model import LogisticRegression

reg = 0.5
clf = LogisticRegression(
    C=1.0 / reg, solver="liblinear", multi_class="auto", random_state=42
)
clf.fit(X_train, y_train)

# make predictions using the test set and calculate the accuracy
y_hat = clf.predict(X_test)

# calculate accuracy on the prediction
acc = np.average(y_hat == y_test)
print("Accuracy is", acc)

run.log("regularization rate", np.float(reg))
run.log("accuracy", np.float(acc))


## Version control your models with the model registry

You can use model registration to store and version your models in your workspace. Registered models are identified by name and version. Each time you register a model with the same name as an existing one, the registry increments the version. Azure Machine Learning supports any model that can be loaded through Python 3.

The code below:

1. Saves the model to disk
1. Uploads the model file to the run 
1. Registers the uploaded model file
1. Transitions the run to a completed state

In [ ]:
import joblib
from azureml.core.model import Model

path = "sklearn_mnist_model.pkl"
joblib.dump(value=clf, filename=path)

run.upload_file(name=path, path_or_stream=path)

model = run.register_model(
    model_name="sklearn_mnist_model",
    model_path=path,
    description="Mnist handwriting recognition",
)

run.complete()

## Deploy the model

The next cell deploys the model to an Azure Container Instance so that you can score data in real-time (Azure Machine Learning also provides mechanisms to do batch scoring). A real-time endpoint allows application developers to integrate machine learning into their apps.

In [ ]:
# create environment for the deploy
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# to install required packages
env = Environment("quickstart-env")
cd = CondaDependencies.create(
    pip_packages=["azureml-dataset-runtime[pandas,fuse]", "azureml-defaults"],
    conda_packages=["scikit-learn==0.22.1"],
)

env.python.conda_dependencies = cd

# Register environment to re-use later
env.register(workspace=ws)

# create config file
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(
    cpu_cores=1,
    memory_gb=1,
    tags={"data": "MNIST", "method": "sklearn"},
    description="Predict MNIST with sklearn",
)

> **Note: The deployment takes around 3 minutes to complete.**

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model

ws = Workspace.from_config()
model = Model(ws, "sklearn_mnist_model")


myenv = Environment.get(workspace=ws, name="quickstart-env", version="1")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

service_name = "sklearn-mnist-svc-" + str(uuid.uuid4())[:4]
service = Model.deploy(
    workspace=ws,
    name=service_name,
    models=[model],
    inference_config=inference_config,
    deployment_config=aciconfig,
)

service.wait_for_deployment(show_output=True)

The [*scoring script*](score.py) file referenced in the code above can be found in the same folder as this notebook, and has two functions:

1. an `init` function that executes once when the service starts - in this function you normally get the model from the registry and set global variables
1. a `run(data)` function that executes each time a call is made to the service. In this function, you normally format the input data, run a prediction, and output the predicted result.

## Test the model service

You can test the model by sending a raw HTTP request to test the web service. 

In [ ]:
# scoring web service HTTP endpoint
print(service.scoring_uri)

In [ ]:
# send raw HTTP request to test the web service.
import requests

# send a random row from the test set to score
random_index = np.random.randint(0, len(X_test) - 1)
input_data = '{"data": [' + str(list(X_test[random_index])) + "]}"

headers = {"Content-Type": "application/json"}

# for AKS deployment you'd need to the service key in the header as well
# api_key = service.get_key()
# headers = {'Content-Type':'application/json',  'Authorization':('Bearer '+ api_key)}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
# print("input data:", input_data)
print("label:", y_test[random_index])
print("prediction:", resp.text)


### View the results of your training

When you're finished with an experiment run, you can always return to view the results of your model training here in the Azure Machine Learning studio:

1. Select **Experiments** (left-hand menu)
1. Select **azure-ml-in10-mins-tutorial**
1. Select **Run 1**
1. Select the **Metrics** Tab

The metrics tab will display the parameter values that were logged to the run.

### View the model in the model registry

You can see the stored model by navigating to **Models** in the left-hand menu bar. Select the **sklearn_mnist_model** to see the details of the model, including the experiment run ID that created the model.

## Clean up resources

If you're not going to continue to use this model, delete the Model service using:

In [ ]:
# if you want to keep workspace and only delete endpoint (it will incur cost while running)
service.delete()

If you want to control cost further, stop the compute instance by selecting the "Stop compute" button next to the **Compute** dropdown.  Then start the compute instance again the next time you need it.


## Next Steps

In this quickstart, you learned how to run machine learning code in Azure Machine Learning.

Now that you have working code in a development environment, learn how to submit a **_job_** - ideally on a schedule or trigger (for example, arrival of new data).

 [**Learn how to get started with Azure ML Job Submission**](GettingStartedWithPythonSDK.ipynb) 